In [4]:
import os 
import cv2 
import numpy as np

In [3]:
def load_images_from_folder(folder):
    images=[]
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))  # Resize image to a fixed size
            images.append(img)
            labels.append(subfolder)
    return images, labels


In [5]:
from sklearn.preprocessing import LabelEncoder


In [6]:
train_folder = "D:/ECG/train"
train_images, train_labels = load_images_from_folder(train_folder)

In [7]:
X_train = np.array(train_images)
y_train = np.array(train_labels)

In [9]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


In [10]:
X_train = X_train / 255.0

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [12]:
y_train = to_categorical(y_train)

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  
])

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train.reshape(-1, 128, 128, 1), y_train, epochs=5, batch_size=32)

Epoch 1/5


29/29 [==============================] - 11s 298ms/step - loss: 1.3832 - accuracy: 0.2985
Epoch 2/5
29/29 [==============================] - 9s 305ms/step - loss: 1.3745 - accuracy: 0.3060
Epoch 3/5
29/29 [==============================] - 9s 312ms/step - loss: 1.3774 - accuracy: 0.3060
Epoch 4/5
29/29 [==============================] - 9s 309ms/step - loss: 1.3742 - accuracy: 0.3060
Epoch 5/5
29/29 [==============================] - 10s 341ms/step - loss: 1.3737 - accuracy: 0.3060


In [16]:
def classify_ecg_image(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise FileNotFoundError("Unable to read the image. Check if the file exists and the path is correct.")
        
        img = cv2.resize(img, (128, 128))  # Resize image to 128x128 pixels
        img = np.array(img) / 255.0  # Normalize pixel values
        img = img.reshape(1, 128, 128, 1)  # Reshape image for model input
        prediction = model.predict(img)
        predicted_class = np.argmax(prediction)
        classes = ['MI', 'History of MI', 'Abnormal heartbeat', 'Normal']
        return classes[predicted_class]
    except Exception as e:
        print("Error:", e)
        return None

image_path = "D:\ECG\ecgimage.jpg"
result = classify_ecg_image(image_path)
if result is not None:
    print("Predicted class:", result)
    


1/1 [==============================] - 0s 144ms/step
Predicted class: Normal
